# New model

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D)
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import (EarlyStopping, ReduceLROnPlateau, ModelCheckpoint)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG16
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

2025-08-14 16:55:06.530934: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-14 16:55:07.623244: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/anubhav_jha/tf-gpu/lib/python3.12/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/hom

## Load and preprocessing data

In [3]:
def load_and_preprocess_data():
    """Load data with improved preprocessing"""
    try:
        X_train = np.loadtxt('input.csv', delimiter=',')
        X_test = np.loadtxt('input_test.csv', delimiter=',')
        Y_train = np.loadtxt('labels.csv', delimiter=',')
        Y_test = np.loadtxt('labels_test.csv', delimiter=',')
    except ValueError:
        import pandas as pd
        X_train = pd.read_csv('input.csv').to_numpy()
        X_test = pd.read_csv('input_test.csv').to_numpy()
        Y_train = pd.read_csv('labels.csv').to_numpy()
        Y_test = pd.read_csv('labels_test.csv').to_numpy()
    
    # Reshape data
    X_train = X_train.reshape(-1, 100, 100, 3)
    X_test = X_test.reshape(-1, 100, 100, 3)
    Y_train = Y_train.reshape(-1, 1)
    Y_test = Y_test.reshape(-1, 1)
    
    # Normalize pixel values
    X_train = X_train.astype('float32') / 255.0
    X_test = X_test.astype('float32') / 255.0
    
    # Per-channel normalization 
    # Compute mean and std per channel on training data
    train_mean = np.mean(X_train, axis=(0, 1, 2))
    train_std = np.std(X_train, axis=(0, 1, 2))
    
    # Apply normalization
    X_train = (X_train - train_mean) / (train_std + 1e-7)
    X_test = (X_test - train_mean) / (train_std + 1e-7)
    
    print(f"Training data shape: {X_train.shape}")
    print(f"Test data shape: {X_test.shape}")
    print(f"Training labels distribution: {np.bincount(Y_train.astype(int).flatten())}")
    
    return X_train, X_test, Y_train, Y_test

# data augmentatio

In [4]:
def create_data_generators(X_train, Y_train, X_test, Y_test):
    """Create data generators with more aggressive augmentation"""
    train_datagen = ImageDataGenerator(
        rotation_range=30,
        width_shift_range=0.3,
        height_shift_range=0.3,
        horizontal_flip=True,
        zoom_range=0.3,
        shear_range=0.2,
        brightness_range=[0.8, 1.2],
        fill_mode='nearest'
    )
    
    # No augmentation for validation data
    val_datagen = ImageDataGenerator()
    
    train_generator = train_datagen.flow(X_train, Y_train, batch_size=32)
    val_generator = val_datagen.flow(X_test, Y_test, batch_size=32, shuffle=False)
    
    return train_generator, val_generator


# CNN architecture

In [5]:
def create_improved_model():
    """Create an improved CNN model with better architecture"""
    model = Sequential([
        # First Convolutional Block
        Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 3)),
        BatchNormalization(),
        Conv2D(32, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Dropout(0.25),
        
        # Second Convolutional Block
        Conv2D(64, (3, 3), activation='relu'),
        BatchNormalization(),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Dropout(0.25),
        
        # Third Convolutional Block
        Conv2D(128, (3, 3), activation='relu'),
        BatchNormalization(),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Dropout(0.25),
        
        # Fourth Convolutional Block
        Conv2D(256, (3, 3), activation='relu'),
        BatchNormalization(),
        Dropout(0.25),
        
        # Global Average Pooling instead of Flatten
        GlobalAveragePooling2D(),
        
        # Dense layers
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    
    return model


# Transfer learning model (VGG16-based)

In [6]:
def create_transfer_learning_model():
    """Create a transfer learning model using VGG16"""
    from tensorflow.keras.applications import VGG16
    from tensorflow.keras.models import Model
    
    # Load pre-trained VGG16 model
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(100, 100, 3))
    
    # Freeze early layers
    for layer in base_model.layers[:-4]:
        layer.trainable = False
    
    # Add custom classifier
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

# Training 

In [7]:
def train_model(model, train_generator, val_generator, model_name="enhanced_model"):
    """Train the model with advanced callbacks"""
    
    # Compile with optimized parameters
    optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)
    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    # Advanced callbacks
    callbacks = [
        EarlyStopping(
            monitor='val_accuracy',
            patience=10,
            restore_best_weights=True,
            verbose=1
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=5,
            min_lr=1e-7,
            verbose=1
        ),
        ModelCheckpoint(
            f'{model_name}_best.h5',
            monitor='val_accuracy',
            save_best_only=True,
            verbose=1
        )
    ]
    
    # Train the model
    history = model.fit(
        train_generator,
        epochs=100,  # More epochs with early stopping
        validation_data=val_generator,
        callbacks=callbacks,
        verbose=1
    )
    
    return history


# Evaluation and visualization

In [8]:
def evaluate_and_visualize(model, X_test, Y_test):
    """Comprehensive model evaluation"""
    
    # Make predictions
    y_pred_prob = model.predict(X_test)
    y_pred = (y_pred_prob > 0.5).astype(int)
    
    # Calculate accuracy
    accuracy = np.mean(y_pred.flatten() == Y_test.flatten())
    print(f"Final Test Accuracy: {accuracy:.4f}")
    
    # Confusion Matrix
    cm = confusion_matrix(Y_test, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['Dog', 'Cat'], yticklabels=['Dog', 'Cat'])
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix - Enhanced Model')
    plt.show()
    
    # Classification Report
    print("\nClassification Report:")
    print(classification_report(Y_test, y_pred, target_names=['Dog', 'Cat']))
    
    return accuracy


# Ensemble method

In [9]:
def create_ensemble_prediction(models, X_test):
    """Create ensemble predictions from multiple models"""
    predictions = []
    
    for model in models:
        pred = model.predict(X_test)
        predictions.append(pred)
    
    # Average predictions
    ensemble_pred = np.mean(predictions, axis=0)
    return (ensemble_pred > 0.5).astype(int)

# Main execution function
def main():
    """Main function to run the entire pipeline"""
    
    print("="*50)
    print("ENHANCED DOG VS CAT CLASSIFICATION")
    print("="*50)
    
    # Step 1: Load and preprocess data
    print("\n1. Loading and preprocessing data...")
    X_train, X_test, Y_train, Y_test = load_and_preprocess_data()
    
    # Step 2: Create data generators
    print("\n2. Creating data generators...")
    train_gen, val_gen = create_data_generators(X_train, Y_train, X_test, Y_test)
    
    # Step 3: Train improved CNN model
    print("\n3. Training improved CNN model...")
    improved_model = create_improved_model()
    print(f"Model parameters: {improved_model.count_params():,}")
    
    history1 = train_model(improved_model, train_gen, val_gen, "improved_cnn")
    acc1 = evaluate_and_visualize(improved_model, X_test, Y_test)
    
    # Step 4: Train transfer learning model
    print("\n4. Training transfer learning model...")
    transfer_model = create_transfer_learning_model()
    print(f"Transfer model parameters: {transfer_model.count_params():,}")
    
    # Recreate generators for transfer learning
    train_gen2, val_gen2 = create_data_generators(X_train, Y_train, X_test, Y_test)
    history2 = train_model(transfer_model, train_gen2, val_gen2, "transfer_learning")
    acc2 = evaluate_and_visualize(transfer_model, X_test, Y_test)
    

# prediction

In [10]:
def create_ensemble_prediction(models, X_test):
    """Create ensemble predictions from multiple models"""
    predictions = []
    
    for model in models:
        pred = model.predict(X_test)
        predictions.append(pred)
    
    # Average predictions
    ensemble_pred = np.mean(predictions, axis=0)
    return (ensemble_pred > 0.5).astype(int)

# Main execution function
def main():
    """Main function to run the entire pipeline"""
    
    print("="*50)
    print("ENHANCED DOG VS CAT CLASSIFICATION")
    print("="*50)
    
    # Step 1: Load and preprocess data
    print("\n1. Loading and preprocessing data...")
    X_train, X_test, Y_train, Y_test = load_and_preprocess_data()
    
    # Step 2: Create data generators
    print("\n2. Creating data generators...")
    train_gen, val_gen = create_data_generators(X_train, Y_train, X_test, Y_test)
    
    # Step 3: Train improved CNN model
    print("\n3. Training improved CNN model...")
    improved_model = create_improved_model()
    print(f"Model parameters: {improved_model.count_params():,}")
    
    history1 = train_model(improved_model, train_gen, val_gen, "improved_cnn")
    acc1 = evaluate_and_visualize(improved_model, X_test, Y_test)
    
    # Step 4: Train transfer learning model
    print("\n4. Training transfer learning model...")
    transfer_model = create_transfer_learning_model()
    print(f"Transfer model parameters: {transfer_model.count_params():,}")
    
    # Recreate generators for transfer learning
    train_gen2, val_gen2 = create_data_generators(X_train, Y_train, X_test, Y_test)
    history2 = train_model(transfer_model, train_gen2, val_gen2, "transfer_learning")
    acc2 = evaluate_and_visualize(transfer_model, X_test, Y_test)
    
    # Step 5: Ensemble prediction
    print("\n5. Creating ensemble prediction...")
    ensemble_pred = create_ensemble_prediction([improved_model, transfer_model], X_test)
    ensemble_acc = np.mean(ensemble_pred.flatten() == Y_test.flatten())
    print(f"Ensemble Accuracy: {ensemble_acc:.4f}")
    
    # Final comparison
    print("\n" + "="*50)
    print("FINAL RESULTS COMPARISON:")
    print("="*50)
    print(f"Improved CNN Accuracy: {acc1:.4f}")
    print(f"Transfer Learning Accuracy: {acc2:.4f}")
    print(f"Ensemble Accuracy: {ensemble_acc:.4f}")
    print("="*50)
    
    return improved_model, transfer_model, ensemble_acc

# Additional utility functions
def plot_training_history(history):
    """Plot training history"""
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

def show_prediction_examples(model, X_test, Y_test, num_examples=8):
    """Show prediction examples"""
    indices = np.random.choice(len(X_test), num_examples, replace=False)
    
    plt.figure(figsize=(16, 8))
    for i, idx in enumerate(indices):
        plt.subplot(2, 4, i+1)
        
        # Denormalize image for display (if normalized)
        img = X_test[idx]
        if img.min() < 0:  # If normalized
            img = (img - img.min()) / (img.max() - img.min())
        
        plt.imshow(img)
        
        pred_prob = model.predict(np.expand_dims(X_test[idx], axis=0))[0][0]
        pred_class = "Cat" if pred_prob > 0.5 else "Dog"
        true_class = "Cat" if Y_test[idx][0] > 0.5 else "Dog"
        
        color = 'green' if pred_class == true_class else 'red'
        plt.title(f'True: {true_class}\nPred: {pred_class} ({pred_prob:.2f})', 
                 color=color)
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()

# Main

In [ ]:
if __name__ == "__main__":
    improved_model, transfer_model, final_accuracy = main()

ENHANCED DOG VS CAT CLASSIFICATION

1. Loading and preprocessing data...


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random

In [ ]:
def simple_prediction_test(model, X_test, Y_test):
    """
    Simple function jo sirf ek random image leta hai aur prediction check karta hai
    No heavy processing, no multiple images - bas ek quick test
    """
    
    # Random image select karo
    idx = random.randint(0, len(X_test) - 1)
    
    # Image prepare karo display ke liye
    test_image = X_test[idx].copy()
    if test_image.min() < 0:
        test_image = (test_image - test_image.min()) / (test_image.max() - test_image.min())
    
    # Model se prediction lo
    prediction = model.predict(X_test[idx:idx+1], verbose=0)[0][0]
    actual_label = Y_test[idx][0]
    
    # Classes determine karo
    if prediction > 0.5:
        predicted_class = "Cat"
        confidence = prediction
    else:
        predicted_class = "Dog" 
        confidence = 1 - prediction
    
    if actual_label > 0.5:
        actual_class = "Cat"
    else:
        actual_class = "Dog"
    
    # Result check karo
    is_correct = (predicted_class == actual_class)
    
    # Simple display
    plt.figure(figsize=(8, 6))
    plt.imshow(test_image)
    
    # Title me sab kuch batao
    if is_correct:
        result_text = f"✅ SAHI HAI!\n"
        title_color = 'green'
    else:
        result_text = f"❌ GALAT HAI!\n"
        title_color = 'red'
    
    result_text += f"Actual: {actual_class}\n"
    result_text += f"Model ne kaha: {predicted_class}\n"
    result_text += f"Confidence: {confidence:.1%}"
    
    plt.title(result_text, fontsize=14, fontweight='bold', color=title_color)
    plt.axis('off')
    plt.tight_layout()
    plt.show()
    
    # Console me bhi print karo
    # print("="*50)
    # if is_correct:
    #     print("🎉 MODEL SAHI HAI!")
    # else:
    #     print("😞 MODEL GALAT HAI!")
    
    print(f"Actual label: {actual_class}")
    print(f"Model prediction: {predicted_class}")
    print(f"Confidence: {confidence:.1%}")
    print(f"Raw score: {prediction:.4f}")
    print("="*50)
    
    return is_correct, predicted_class, actual_class, confidence


def quick_test_multiple_times(model, X_test, Y_test, num_tests=5):
    """
    Agar aap chahte hain to multiple times test kar sakte hain
    """
    correct_count = 0
    
    print(f"\n🔄 {num_tests} random images test kar rahe hain...\n")
    
    for i in range(num_tests):
        print(f"Test {i+1}:")
        is_correct, pred, actual, conf = simple_prediction_test(model, X_test, Y_test)
        if is_correct:
            correct_count += 1
        print()  # Empty line for spacing
    
    accuracy = correct_count / num_tests
    print(f"📊 RESULT: {correct_count}/{num_tests} sahi ({accuracy:.1%} accuracy)")


# =============================================================================
# USAGE - Sirf ye line run karni hai
# =============================================================================

# Check karo ki model aur data available hai
try:
    # Ye automatically detect karega available model
    available_models = []
    for name in ['transfer_model', 'improved_model', 'model', 'best_model']:
        if name in globals() and globals()[name] is not None:
            available_models.append((name, globals()[name]))
    
    if not available_models:
        print("❌ Koi model nahi mila!")
        print("Pehle apna model load/train karo")
    elif 'X_test' not in globals() or 'Y_test' not in globals():
        print("❌ Test data nahi mila!")
        print("X_test aur Y_test load karo")
    else:
        # Best model use karo
        model_name, model = available_models[0]
        print(f"✅ Using model: {model_name}")
        print(f"✅ Test data: {len(X_test)} images available")
        
        print("\n" + "="*50)
        print("🎯 EK RANDOM IMAGE TEST")
        print("="*50)
        
        # Sirf ek test
        simple_prediction_test(model, X_test, Y_test)
        
        # Agar aur test karna hai to uncomment karo
        # quick_test_multiple_times(model, X_test, Y_test, num_tests=3)

except Exception as e:
    print(f"❌ Error: {e}")
    print("\nSetup check karo:")
    print("1. Model trained hai?")
    print("2. X_test, Y_test loaded hai?")

print("\n💡 Agar aur test karna hai to run karo:")
print("simple_prediction_test(your_model, X_test, Y_test)")

❌ Koi model nahi mila!
Pehle apna model load/train karo

💡 Agar aur test karna hai to run karo:
simple_prediction_test(your_model, X_test, Y_test)
